In [48]:
import os
import pandas as pd
import numpy as np
import subprocess

In [49]:
df = pd.read_csv('fails2.csv')
df_sample = df.head(n=100)

In [50]:
df_sample['Test Dir']

0     aapg_iclass_rv64imafdc_imc_bringup_08072020159...
1     aapg_iclass_rv64imafdc_imc_bringup_08072020159...
2     aapg_iclass_rv64imafdc_imc_bringup_08072020159...
3     aapg_iclass_rv64imafdc_imc_bringup_08072020159...
4     aapg_iclass_rv64imafdc_imc_bringup_08072020159...
                            ...                        
95    aapg_iclass_rv64imafdc_imafd_bringup_080720201...
96    aapg_iclass_rv64imafdc_imafd_bringup_080720201...
97    aapg_iclass_rv64imafdc_imafd_bringup_080720201...
98    aapg_iclass_rv64imafdc_imafd_bringup_080720201...
99    aapg_iclass_rv64imafdc_imafd_bringup_080720201...
Name: Test Dir, Length: 100, dtype: object

In [51]:
final_df = pd.DataFrame({
    'test_dir': [None],
    'fail_reason': [None],
    'diff_disass': [None]
})

In [52]:
i = 0
import re
os.chdir('/Users/abishek/Desktop/IIT_M_Winter/verification')
for name in df_sample['Test Dir']:
    grep_check = None
    reason = None
    diff_disass = None
    cur_dir = os.getcwd()
    os.chdir(os.path.join('8_july_wrk/aapg/p',name))
    os.system('source /Users/abishek/Desktop/IIT_M_Winter/verification/comp_script.sh spike.dump rtl.dump > temp.txt')
    out = ""
    with open('temp.txt','r') as f:
        while(1):
            line = f.readline()
            if not line:
                break
            out = out + line + '\n'
    #print(out)
    diff_disass = out
    os.remove('temp.txt')
    
    if(out==""):
        #print('empty diff')
        # Read spike last line?
        if(os.path.exists('spike.dump')):
            with open('spike.dump','r') as sf:
                for line in sf.readlines()[-1:]:
                    x = re.findall("8.*", line)
                    x = x[0][:8]
                    grep_check = x
                    reason = "rtl hang due to -->"
    else:
        x = re.findall("8.*", out)
        x = x[0][:8]
        grep_check = x
    if grep_check != None and reason==None:
        cmd = 'grep '+grep_check+' '+name+'.disass > temp2.txt'
        os.system(cmd)
        out = ""
        with open('temp2.txt','r') as f:
            while(1):
                line = f.readline()
                if not line:
                    break
                out = out + line + '\n'
        x = re.findall("8.*:\t.*\n", out)
        if(len(x)!=0):
            x = x[0]
        else:
            x = ''
        reason = "mismatch due to --> " + x
        os.remove('temp2.txt')
    elif grep_check != None and reason != None:
        #print('here')
        cmd = 'grep -C 2 '+grep_check+' '+name+'.disass > temp2.txt'
        os.system(cmd)
        out = ""
        with open('temp2.txt','r') as f:
            while(1):
                line = f.readline()
                if not line:
                    break
                out = out + line + '\n'
        diff_disass = out
        #print(diff_disass)
        x = re.findall(grep_check+'.*\n', out)
        if(len(x)!=0):
            x = x[0]
        else:
            x = ''
        reason = reason + x
        #print(reason)
        os.remove('temp2.txt')
        
    
    df_temp = pd.DataFrame({
        'test_dir': [name],
        'fail_reason': [reason],
        'diff_disass': [diff_disass]
    })
    final_df = final_df.append(df_temp)
    #print(df_temp)
    os.chdir(cur_dir)


In [53]:
#print(final_df)
final_df.to_csv('final_df.csv',index=False)

In [ ]:
#     try:
#         bin_string = subprocess.check_output('source /Users/abishek/Desktop/IIT_M_Winter/verification/comp_s spike.dump rtl.dump', shell=True,stderr=subprocess.STDOUT)
#     except subprocess.CalledProcessError as e:
#         raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))
#     print(bin_string)